In [129]:
from tensorflow import keras
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import cv2
from keras.applications import VGG16
from keras.layers import Flatten, Dense, Dropout, LeakyReLU, Input, concatenate, add, Reshape, Conv2D, Activation
from keras.optimizers import Adam, RMSprop, Adagrad, SGD
from keras.models import Model, load_model
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.utils import to_categorical, plot_model
from glob import glob
from sklearn.model_selection import train_test_split
from skimage.io import imread
from skimage.transform import resize
import collections
import os
from scipy.spatial.distance import cosine
from tensorflow.keras.preprocessing.image import ImageDataGenerator as ImageDataGenerator
from matplotlib.pyplot import figure, imshow, axis
from matplotlib.image import imread
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
import seaborn as sns
from keras.applications.vgg16 import VGG16

In [130]:
# Define parameters
ROLLNUMBER = 4228
NUM_CLASSES = 6
labels_dict = {"freeway":0,"mountain":1,"palace":2,"river":3,"ship":4,"stadium":5}
# Batch size is a hyperparameter, feel free to change this. You may also change the number of epochs.
BATCH_SIZE = 32
EPOCHS = 1

In [131]:
def DML():
    inputs1 = layers.Input((1024, 1))
    inputs2 = layers.Input((1024, 1))
    inputs1_r = layers.Reshape((32, 32, 1))(inputs1)
    inputs2_r = layers.Reshape((32, 32, 1))(inputs2)
    merged =  layers.Concatenate(axis=-1)([inputs1_r, inputs2_r])
    conv2d_1 = layers.Conv2D(16, (3, 3), padding = "same")(merged)
    conv2d_2 = layers.Conv2D(16, (3, 3), padding = "same")(conv2d_1)
    conv2d_3 = layers.Conv2D(16, (3, 3), padding = "same")(conv2d_2)
    added = tf.keras.layers.Add()([conv2d_3, conv2d_1])
    conv2d_4 = layers.Conv2D(16, (3, 3), strides=(2,2), padding = "same")(added)
    conv2d_5 = layers.Conv2D(16, (3, 3), padding = "same")(conv2d_4)
    conv2d_6 = layers.Conv2D(16, (3, 3), strides=(2,2),padding = "same")(added)
    added_2 = layers.Add()([conv2d_5, conv2d_6])
    leaky = layers.LeakyReLU(alpha=0.1)(added_2)
    flatten = layers.Flatten()(leaky)
    x = layers.Dense(1024)(flatten)
    x = layers.LeakyReLU(alpha=0.1)(x)
    x = layers.Dense(512)(x)
    x = layers.LeakyReLU(alpha=0.1)(x)
    x = layers.Dense(64)(x)
    x = layers.LeakyReLU(alpha=0.1)(x)
    x = layers.Dense(1)(x)
    output = layers.Activation('softmax')(x)
    model = Model(inputs=[inputs1, inputs2], outputs=output)
    # compile model
    model.compile(optimizer = Adam(lr= 0.001), 
                   loss ='categorical_crossentropy',
                   metrics = ['accuracy'])
    # get model summary
    model.summary()
    plot_model(model, to_file='model_plot.png', show_shapes=True)
    return model

In [132]:
dml = DML()

Model: "model_24"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_53 (InputLayer)           [(None, 1024, 1)]    0                                            
__________________________________________________________________________________________________
input_54 (InputLayer)           [(None, 1024, 1)]    0                                            
__________________________________________________________________________________________________
reshape_50 (Reshape)            (None, 32, 32, 1)    0           input_53[0][0]                   
__________________________________________________________________________________________________
reshape_51 (Reshape)            (None, 32, 32, 1)    0           input_54[0][0]                   
___________________________________________________________________________________________

In [133]:
dml_train_df = pd.read_csv('dml_train.csv')
dml_test_df = pd.read_csv('dml_test.csv')
dml_train_df.head()
train_samples = dml_train_df.shape[0]
test_samples = dml_test_df.shape[0]
dml_train_df.shape

(33600, 3)

In [152]:
def dml_data_generator(df, batch_size = 32, mode='train'):
    X1 = []
    X2 = []
    y = []
    for index, row in df.iterrows():
        #print(row['c1'], row['c2'])
        sat_vector = np.load(row['sat_path']).reshape(1024,1)
        str_vector = np.load(row['str_path']).reshape(1024,1)
        X1.append(sat_vector)
        X2.append(str_vector)
        y.append(row['label'])
    samples_per_epoch = len(X1)
    number_of_batches = samples_per_epoch/batch_size
    counter=0

    while 1:

        X1_batch = np.array(X1[batch_size*counter:batch_size*(counter+1)]).astype('float32')
        X2_batch = np.array(X2[batch_size*counter:batch_size*(counter+1)]).astype('float32')
        y_batch = np.array(y[batch_size*counter:batch_size*(counter+1)]).astype('float32')
        counter += 1
        yield [X1_batch, X2_batch], y_batch

        #restart counter to yeild data in the next epoch as well
        if counter >= number_of_batches:
            counter = 0

In [153]:
dml_train_gen = dml_data_generator(dml_train_df,batch_size=BATCH_SIZE)
dml_test_gen = dml_data_generator(dml_test_df,batch_size=BATCH_SIZE)

In [154]:
earlystop = EarlyStopping(patience = 10)
learning_rate_reduction = ReduceLROnPlateau(monitor ='val_acc', patience = 2, verbose = 1, factor = 0.5, min_lr = 0.001)
checkpoint = ModelCheckpoint("Results.h5", monitor = 'val_acc', verbose=1, save_best_only = True, save_weights_only = False, mode = 'auto', period = 1)
callback = [checkpoint, earlystop, learning_rate_reduction]
# select to train on cpu
dml.fit_generator(dml_train_gen, steps_per_epoch = train_samples // BATCH_SIZE, validation_data = dml_test_gen, 
                    validation_steps = test_samples // BATCH_SIZE, epochs = EPOCHS, callbacks = callback)

UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node model_24/conv2d_167/Conv2D (defined at <ipython-input-154-b85315ec5506>:7) ]] [Op:__inference_train_function_10161]

Function call stack:
train_function


In [155]:
def sort_by_value(dict_obj):
    return collections.OrderedDict(sorted(dict_obj.items(), key=lambda kv: kv[1]))

In [156]:
def get_similar(query_fv_path,database,metric='dml',query_view='str',batch_size=32):
    results = None
    
    if query_view == 'sat':
        pass
    elif query_view == 'str':
        pass

    return sort_by_value(results)

In [159]:
Database = pd.read_csv('Retrieval_Database.csv')
Database.head()

,img_path,view,class,fv_path
0,SatView/Images/stadium/stadium_285.jpg,sat,stadium,SatView/Features/stadium_285.npy
1,SatView/Images/ship/ship_191.jpg,sat,ship,SatView/Features/ship_191.npy
2,SatView/Images/ship/ship_070.jpg,sat,ship,SatView/Features/ship_070.npy
3,StreetView/Images/stadium/33080359_333f0e95af_...,str,stadium,StreetView/Features/33080359_333f0e95af_o.npy
4,StreetView/Images/palace/4822142388_483a9e454f...,str,palace,StreetView/Features/4822142388_483a9e454f_o.npy
